<a href="https://colab.research.google.com/github/takatakamanbou/ML/blob/2024/ML2024_ex06notebookA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ML ex06notebookA

<img width=72 src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/ML/ML-logo.png"> [この授業のウェブページ](https://www-tlab.math.ryukoku.ac.jp/wiki/?ML/2024)


----
## ニューラルネットワークと深層学習 (2)
----

ひとつ前の notebook では，ニューラルネットワークとはどういうものか，ということを説明しました．この notebook では，階層型ニューラルネットワークの学習の方法について説明します．また，簡単な回帰の問題への適用例を示します．





----
### 準備

以下，コードセルを上から順に実行してながら読んでいってね．

In [ ]:
# 準備あれこれ
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation, rc  # アニメーションのため
import seaborn
seaborn.set()

# 深層学習フレームワーク PyTorch のため
import torch
import torch.nn as nn

---
### 階層型ニューラルネットワークの学習

階層型ニューラルネットワークのパラメータは，通常，教師あり学習によって決定されます．教師あり学習ですので，「ネットワークの出力とその正解の値との間のずれ」の大きさを表す量を定式化し，それが小さくなるようにパラメータを調節します．この「ずれ」を表す関数を，**損失関数**(loss function)と呼びます．

平面あてはめの最小二乗法では，「モデル出力とその正解の値との間の二乗誤差」を最小化するという，似たような話がありました．また，ロジスティック回帰では，「モデル出力と正解との間の交差エントロピー」を最小化していました．
ニューラルネットワークの場合，損失関数としては，二乗誤差・交差エントロピーのどちらも使用することができます．ニューラルネットワークを回帰問題に適用する場合には二乗誤差が，識別問題に適用する場合には交差エントロピーがよく用いられます（注）．

損失関数の最適化（最小化）は，ロジスティック回帰と同様，**最急降下法**などの**勾配法**が用いられます．
ニューラルネットワークのモデルは複雑な式をしていますが，損失関数のパラメータに関する勾配を求めることが可能です．というか，ニューラルネットワークモデルは，勾配を計算可能なように微分可能な要素の組み合わせで作られています．


<span style="font-size: 75%">
※注: 問題によって，これ以外にも様々な損失関数が用いられます．
</span>


#### 例: 中間層が一つで損失関数が二乗誤差の場合

<img width="40%" src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/ML/neuralnet2.png" align="right">

ニューラルネットワークモデルの構造と損失関数を定めれば，勾配法でモデルのパラメータを学習させることができます．
ここでは，図のように，中間層を一つもち，入力が$D$次元，中間層および出力層のニューロン数がそれぞれ $H$，$M$ のニューラルネットワークを考えます．
このネットワークの出力 $z_m$ は次式で求まります．

$$
\begin{aligned}
y_h &= \sigma\left( v_{h,0} + \sum_{d=1}^{D}v_{h,d}x_d\right) & (h = 1, 2, \ldots, H) \qquad (1)\\
z_m &= \sigma\left(w_{m,0} + \sum_{h=1}^{H}w_{m,h}y_h \right) & (m=1,2,\ldots,M) \qquad (2)
\end{aligned}
$$

損失関数に二乗誤差を用いるものとして，その勾配を式で表してみます．

学習データが $N$ 個与えられるとして，出力の正解を $\widetilde{z}_{n,m}$ ($n=1,2,\ldots,N$) とおくと，二乗誤差の損失関数 $L$ は

$$
L = \frac{1}{2}\sum_{n=1}^{N}\sum_{m=1}^{M}(\widetilde{z}_{n,m} - z_{n,m})^2
$$

となります．ここで，$\ell_{n} = \frac{1}{2}\sum_{m=1}^{M}(\widetilde{z}_{n,m} - z_{n,m})^2$ とおけば， $L = \sum_{n=1}^{N}\ell_n$ より，$L$ の勾配は $\ell_n$ の勾配の和となります．
簡単のため添字 $n$ を省略して $\ell$ の勾配を計算すると，次のようになります．

$$
\begin{aligned}
\frac{\partial \ell}{\partial w_{m,h}} &= -(\widetilde{z}_m - z_m)\sigma'(z_m)y_h & (3)\\
\frac{\partial \ell}{\partial v_{h,d}}
&= -\left(\sum_{m=1}^{M}(\widetilde{z}_m - z_m) \sigma'(z_m)w_{m,h}\right)\sigma'(y_h)x_d & (4)
\end{aligned}
$$

これらの式の導出過程や記号$\sigma'$の意味は「［発展］損失関数の勾配の導出」節を参照してください．勾配の導出は，ロジスティック回帰のときと同様に，高校数学（合成関数の微分）＆大学初年次の数学（偏微分）の知識があればできます．

#### ［発展］損失関数の勾配の導出

$\ell_n = \frac{1}{2}\sum_{m=1}^{M}(\widetilde{z}_{n,m} - z_{n,m})^2$ に対する勾配 $\frac{\partial \ell_n}{\partial v_{h,d}}$ と $\frac{\partial \ell_n}{\partial w_{h}}$ を求めます．
これらが求まれば，$L=\sum_{n=1}^{N}\ell_n$ より，$\frac{\partial L}{\partial v_{h,d}} = \sum_{n=1}^{N}\frac{\partial \ell_n}{\partial v_{h,d}}$  および $\frac{\partial L}{\partial w_{h}} = \sum_{n=1}^{N}\frac{\partial \ell_n}{\partial w_{h}}$ となります．

以下では，式を見やすくするため，添字 $n$ も適宜省略しています．


まず，準備として $\frac{\partial z_m}{\partial w_{m,h}}$ および
$\frac{\partial z_m}{\partial v_{h,d}}$ を求めます．
式(1),(2)より，次のように求まります．

$$
\begin{aligned}
\frac{\partial z_m}{\partial w_h} &= \sigma'(z_m)y_h \qquad
\frac{\partial y_h}{\partial v_{h,d}} = \sigma'(y_h)x_d\\
\frac{\partial z_m}{\partial v_{h,d}} &= \sigma'(z_m)\frac{\partial }{\partial v_{h,d}}\left( w_{m,0} + \sum_{h=1}^{H}w_{m,h}y_h \right)\\
&= \sigma'(z_m) w_{m,h} \frac{\partial y_h}{\partial v_{h,d}} = \sigma(z_m)w_{m,h}\sigma'(y_h)x_d
\end{aligned}
$$

ここで，$\sigma'$ というのは，$\frac{d\sigma(s)}{ds}$ を $\sigma(s)$ 自身で表したものです．例えばシグモイド関数の場合は，$\sigma' = \sigma(1-\sigma)$ より，$\sigma'(z_m) = z_m(1-z_m)$ などとなります（2クラス識別のロジスティック回帰の学習アルゴリズム導出過程参照）．



これらの式から，勾配 $\frac{\partial \ell}{\partial w_{m,h}}$ と $\frac{\partial \ell}{\partial v_{h,d}}$ は次式のように求まります．
$$
\begin{aligned}
\frac{\partial \ell}{\partial w_{m,h}} &= (\widetilde{z}_m - z_m)\left( - \frac{\partial z_m}{\partial w_{m,h}} \right) = -(\widetilde{z}_m - z_m)\sigma'(z_m)y_h\\
\frac{\partial \ell}{\partial v_{h,d}} &= \sum_{m=1}^{M}(\widetilde{z}_m - z_m)\left( - \frac{\partial z_m}{\partial v_{h,d}} \right) \\
&= -\left(\sum_{m=1}^{M}(\widetilde{z}_m - z_m) \sigma'(z_m)w_{m,h}\right)\sigma'(y_h)x_d
\end{aligned}
$$


#### 誤差逆伝播

階層型ニューラルネットワークのパラメータを勾配法で学習させる方法のことを，**誤差逆伝播法**(error backpropagation)といいます．

ニューラルネットワークの出力の計算の際には， ネットワークへ入力された値が，最初の中間層 → … → 最後の中間層 → 出力層と順に伝わっていきます．
これに対して，勾配の計算では，出力層で求めた誤差の値が，出力層 → 最後の中間層 → … 最初の中間層と，ネットワークを逆向きに伝わっていきます．
そのため，「誤差が逆向きに伝播する」ということで，誤差逆伝播と呼ばれています．

参考までに，式(3),(4)を変形して，「誤差が逆向きに伝播する」ことを確認してみます．

式(3)において，$\varepsilon_{m} = (\widetilde{z}_m - z_m)\sigma'(z_m)$ とおくと，

$$
\frac{\partial \ell}{\partial w_{m,h}} = -(\widetilde{z}_m - z_m)\sigma'(z_m)y_h = -\varepsilon_{m}y_h\\
$$

と表わせます．このとき，式(4)の方は

$$
\begin{aligned}
\frac{\partial \ell}{\partial v_{h,d}}
&= -\left(\sum_{m=1}^{M}(\widetilde{z}_m - z_m) \sigma'(z_m)w_{m,h}\right)\sigma'(y_h)x_d \\
&= -\left(\sum_{m=1}^{M} w_{m,h} \varepsilon_{m} \right)\sigma'(y_h)x_d
\end{aligned}
$$

となります．$\varepsilon_{m}$を「$m$番目の出力層ニューロンの誤差」とみなすと，中間層ニューロンの勾配は，それらを重み $w_{m,h}$ を介して逆向きに伝えたものから計算できることが分かります．

中間層が複数ある場合は，このような過程を出力層側から入力側へと一層ずつ繰り返すことになります．

---
### 具体例: 非線形回帰



#### 問題設定
「汎化と過適合」の回の notebook で多項式あてはめしてみていた例をニューラルネットワークにやらせてみましょう．
以下のグラフに描かれた8つの点のX座標を入力，Y座標を出力の正解とする回帰問題です．

In [ ]:
# データを用意
X = np.linspace(-3, 4, num=8)
Y = np.sin(X)  # 真の関係は y = sin(x)
xmin, xmax = -4.5, 5.5
Xr =  np.linspace(xmin, xmax, num=100)
Yr = np.sin(Xr)
# グラフを描く
fig, ax = plt.subplots(1, facecolor='white', figsize=(6, 4))
ax.scatter(X, Y)
ax.plot(Xr, Yr, color='gray')
ax.set_xlim(xmin, xmax)
ax.set_ylim(-1.5, 1.5)
plt.show()

<img width="40%" src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/ML/neuralnet4.png" align="right">

このような問題のためのニューラルネットワークとして最も単純なものは，図のように，中間層が一層で出力層にニューロンをひとつだけ含む形のものです．このニューラルネットの入出力は次式で表されます．

$$
\begin{aligned}
y_h &= \sigma\left( v_{h,0} + v_{h,1}x\right) \qquad (h = 1, 2, \ldots, H) \\
z &= w_{0} + \sum_{h=1}^{H}w_{h}y_h
\end{aligned}
$$

一般に回帰の問題では，出力の値の範囲を限定しない方が適切であるため，出力層ニューロンには活性化関数を用いません（注）．
また，中間層ニューロンの活性化関数にはシグモイド関数を用いることにします．

<span style="font-size: 75%">
※注: 「活性化関数を恒等関数 $\sigma(s) = s$ としている」という方がより適切．
</span>

損失関数には二乗誤差を用いることにします．$N$ 個の学習データのうち $n$ 番目のものを入力したときのネットワークの出力を $z_n$，その正解（上記のY座標の値）を $\widetilde{z}_n$ とおいて，損失関数を $L$ と表すことにすると，

$$
L = \frac{1}{2}\sum_{n=1}^{N}(\widetilde{z}_n - z_n)^2
$$

です．







#### 実験


上記の問題について実際にコードを実行して，ニューラルネットに学習させてみましょう．

ここでは，[PyTorch](https://pytorch.org/) という深層学習フレームワーク（注）を用いてコードを書いています．
深層学習フレームワークというのは，ニューラルネットワークの学習などの処理を簡単なコードで実装できるようにしたソフトウェアの集まり（ライブラリ等とも呼ばれる）のことです．PyTorch の他にも， Google の [TensorFlow](https://www.tensorflow.org/) などいくつか有名なものがあります．複雑なニューラルネットワークを簡単に実装できます．
例えば，以下のコードでは，ニューラルネットワークの構造を定義して，利用する損失関数を指定するだけで，勾配の計算は PyTorch が自動的にやってくれています．


In [ ]:
# PyTorch を用いたニューラルネットのクラスの定義
#
class NeuralNetR(nn.Module):

    # コンストラクタ．
    #   D: 入力次元数，H: 中間層ニューロン数，D: 出力層ニューロン数
    def __init__(self, D, H, M):
        super(NeuralNetR, self).__init__()
        self.layer1 = nn.Linear(D, H) # 入力 → 中間層
        self.sigmoid = nn.Sigmoid()   # 中間層の活性化関数
        self.layer2 = nn.Linear(H, M) # 中間層 → 出力層
        # 以下ではデモのためあえて過適合を起こしやすい初期値にしている
        nn.init.uniform_(self.layer2.weight, -10, 10)

    # 入力 X に対するネットワークの出力を計算
    def forward(self, X):
        Y = self.sigmoid(self.layer1(X))
        Z = self.layer2(Y)
        return Z


# データの準備
X = np.linspace(-3, 4, num=8, dtype=np.float32) # 学習データ（入力）
Zt = np.sin(X)                                  # 学習データ（出力の正解）
X_tensor = torch.from_numpy(X[:, np.newaxis])
Zt_tensor = torch.from_numpy(Zt[:, np.newaxis])
xmin, xmax = -4.5, 5.5
Xr = np.linspace(xmin, xmax, num=100, dtype=np.float32)
Xr_tensor = torch.from_numpy(Xr[:, np.newaxis])

In [ ]:
#@title #### ニューラルネットワークによる非線形回帰
#@markdown このセルを実行すると，再生ボタンなどが付いたグラフが現れます（**少し時間かかるかも**）．以下のメニューから中間層ニューロン数$H$を変えられるよ．
H = 10 #@param [10, 100] {type: 'raw'}

# 学習の準備
model = NeuralNetR(1, H, 1) # モデルのインスタンスを生成
loss_func = nn.MSELoss()    # 平均二乗誤差を損失関数とする
optimizer = torch.optim.SGD(model.parameters(), lr=0.01) # 最適化法を選ぶ
nitr = 2000

# グラフの準備
fig = plt.figure(facecolor='white', figsize=(12, 6))
ax1 = fig.add_subplot(121)
ax1.set_xlim(xmin, xmax)
ax1.set_ylim(-3, 3)
ax1.scatter(X, Zt)
ax1.plot(Xr, np.sin(Xr), color='gray')
ax2 = fig.add_subplot(122)
ax2.set_xlim(0, nitr)
ax2.set_ylim(0, 1)
iList = []
mseList = []
aList = []

for i in range(nitr+1):

    Z = model(X_tensor) # 出力を計算
    loss = loss_func(Z, Zt_tensor) # 損失関数の値を計算
    optimizer.zero_grad()
    loss.backward() # 勾配を計算
    optimizer.step() # パラメータを更新

    if (i < 100 and i % 10 == 0) or i % 100 == 0:
        #print(i, loss.item())
        Zr = model(Xr_tensor).detach().numpy()
        a1 = ax1.plot(Xr, Zr, color='red')
        iList.append(i)
        mseList.append(loss.item())
        a2 = ax2.plot(iList, mseList, color='blue', marker='.')
        aList.append(a1+a2)

anim = animation.ArtistAnimation(fig, aList, interval=300)
rc('animation', html='jshtml')
plt.close()
anim

##### ★★ やってみよう ★★

中間層のニューロン数 `H=10` と `H=100` それぞれの条件で，上記のセルを複数回実行して，学習の様子を観察しましょう．
中間層ニューロン数の違いでどのような違いが生じているか考えて，紙媒体にメモしておきましょう．